# Problem:

Using user review data from FourSquare, our aim was to develop a recommendation system to provide a <span style="text-decoration:underline">new restaurant suggestion</span> for New York, NY where users might like. Given the diverse application of this problem, I wanted to learn how to develop and implement such system using machine learning. Below, we graphically illustrate our project goal.

Currently there is a gap in the restaurant recommendation applications market.  Virtually no application offers users the possibility of creating an account in order to receive proper recommendations based on his tastes or previous experiences.  There are, however, applications focused on a niche segments like giving users the possibility of reviewing places or getting them based on how expressive they are, but at the end of the day these applications are nothing more than aggregators.

Many applications on this segment like Zomato Yelp have the ability of rating, reviewing and how it is getting the information to the end user.

These applications take advantage of the fact that users increasingly place their trust in reviews that are coming from more people and some of them offer the possibility of creating an account and keeping track of submitted reviews, ratings and past experiences.  However, not one of them can or would recommend users places based on their past experiences.

# Data:
The Foursquare data is vast and rich: it contains millions of reviews of different business types (e.g. restaurants and dry cleaners), and each business type has different set of attributes associated with them. Each review at minimum consists of review text and review star rating. Because of its size and richness, it presented an initial challenge for us to decide what data to include in our modeling and what features should be engineered. In addition, not every user has made several reviews or reviews with different ratings, so we needed to figure out what datasets to use for training vs. testing. To this end, we performed Exploratory Data Analysis(EDA) on the data, specifically to look at relevant business attributes, user attributes, and reviews. We reasoned that these are the core information needed to link users and businesses with their preferences (hence, we disregarded data such as checkin, photos, and tip).

## Motivation:

The  concept  of  recommender  systems  generally  grows  out  of the  idea  of  information  reuse and  persistent  preferences.  It  is  an  idea  that  does  not  begin  with  computers  and  technology. 
It’s  an  idea that  one  can  find  in  cavemen,  ants  and  other creatures  too.  We  may  have  seen  ants  running  around  in  our house. The ants follow in a line from the ants that went before and found food. This is because ants have genetically evolved to  leave  markers  for  other  ants.  These  markers  serve  as  a recommender  to  other  ants,  showing  them  the  way  to  food. The similar scenario can be seen in humans. People are more likely  to  follow  something  if  majority  of  other  users  have liked  and  done  that  particular  thing. Thus  the motivation  for these projects comes  from  the  fact  that  in  today’s  world recommending an item to a user has gained much importance and  popularity  as  well.  Sometimes  users  have  less  time  to browse  a  site  and  are  looking  for  quick  recommendation  of products  that  are  hot trending  or  which  they  would  probably like.  Also  some  users,  quite  acceptably  are  confused  when they see a long list of item. They are in a fix
whether he/she would  like  this  or  that  one.  Thus  recommender  systems  can keep  a  track  of  each  user’s   taste/likes   and   accordingly recommend specific items to specific users.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


##  Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`

In [2]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [3]:
latitude = 40.7308619
longitude = -73.9871558
CLIENT_ID = 'QMAJMFICCDBVSWUHPSBBALBKYGVWJQOAEF11DA4BNOBZ23XO' # your Foursquare ID
CLIENT_SECRET = '3JN54DEWNJNVNIPYUV0K4FUSIK02OJMB0S4CFMZEYRIIFKMV' # your Foursquare Secret
categoryId = '4d4b7105d754a06374d81259'
VERSION = '20180604'
section='food'

In [33]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,categoryId)
url	

'https://api.foursquare.com/v2/venues/search?client_id=QMAJMFICCDBVSWUHPSBBALBKYGVWJQOAEF11DA4BNOBZ23XO&client_secret=3JN54DEWNJNVNIPYUV0K4FUSIK02OJMB0S4CFMZEYRIIFKMV&ll=40.7308619,-73.9871558&v=20180604&categoryId=4d4b7105d754a06374d81259'

In [10]:
import requests

In [35]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'shortName': 'Deli / Bodega', 'pluralName': ...",83328,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/katzs-delicatess...,False,4acbe67af964a52044c820e3,205 E Houston St,US,New York,United States,at Ludlow St,968,"[205 E Houston St (at Ludlow St), New York, NY...",NaN,40.722161,-73.987213,Lower East Side,10002,NY,Katz's Delicatessen,v-1545944251,36639778
1,"[{'shortName': 'Food Court', 'pluralName': 'Fo...",682503,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/two-tablespoons-...,False,594277b586f4cc0f251fc389,445 Albee Sq,US,Brooklyn,United States,NaN,4426,"[445 Albee Sq, Brooklyn, NY 11201, United States]","[{'lng': -73.98257926844987, 'lat': 40.6912497...",40.691250,-73.982579,NaN,11201,NY,DeKalb Market Hall,v-1545944251,NaN
2,"[{'shortName': 'Coffee Shop', 'pluralName': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f5e82dae4b01989c438cbb6,1 Rockefeller Plz,US,New York,United States,at W 49th St,3163,"[1 Rockefeller Plz (at W 49th St), New York, N...","[{'lng': -73.978681, 'lat': 40.758545, 'label'...",40.758545,-73.978681,NaN,10020,NY,Blue Bottle Coffee,v-1545944251,NaN
3,"[{'shortName': 'Food Court', 'pluralName': 'Fo...",402624,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/mimis-hummus-urb...,False,55c2627a498e6734787e29ef,230 Park Avenue,US,New York,United States,btwn E 45th St & Vanderbilt Ave,2763,[230 Park Avenue (btwn E 45th St & Vanderbilt ...,"[{'lng': -73.97689591949668, 'lat': 40.7544345...",40.754435,-73.976896,NaN,10017,NY,UrbanSpace Vanderbilt,v-1545944251,138447335
4,"[{'shortName': 'Market', 'pluralName': 'Market...",NaN,NaN,NaN,NaN,NaN,NaN,False,579c12cd498e6e815b3acf50,101 Liberty St,US,New York,United States,btwn Church & Greenwich St,3120,"[101 Liberty St (btwn Church & Greenwich St), ...","[{'lng': -74.011976, 'lat': 40.710075, 'label'...",40.710075,-74.011976,NaN,10007,NY,Eataly Downtown,v-1545944251,NaN


In [36]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,id
0,Katz's Delicatessen,"[{'shortName': 'Deli / Bodega', 'pluralName': ...",205 E Houston St,US,New York,United States,at Ludlow St,968,"[205 E Houston St (at Ludlow St), New York, NY...",NaN,40.722161,-73.987213,Lower East Side,10002,NY,4acbe67af964a52044c820e3
1,DeKalb Market Hall,"[{'shortName': 'Food Court', 'pluralName': 'Fo...",445 Albee Sq,US,Brooklyn,United States,NaN,4426,"[445 Albee Sq, Brooklyn, NY 11201, United States]","[{'lng': -73.98257926844987, 'lat': 40.6912497...",40.691250,-73.982579,NaN,11201,NY,594277b586f4cc0f251fc389
2,Blue Bottle Coffee,"[{'shortName': 'Coffee Shop', 'pluralName': 'C...",1 Rockefeller Plz,US,New York,United States,at W 49th St,3163,"[1 Rockefeller Plz (at W 49th St), New York, N...","[{'lng': -73.978681, 'lat': 40.758545, 'label'...",40.758545,-73.978681,NaN,10020,NY,4f5e82dae4b01989c438cbb6
3,UrbanSpace Vanderbilt,"[{'shortName': 'Food Court', 'pluralName': 'Fo...",230 Park Avenue,US,New York,United States,btwn E 45th St & Vanderbilt Ave,2763,[230 Park Avenue (btwn E 45th St & Vanderbilt ...,"[{'lng': -73.97689591949668, 'lat': 40.7544345...",40.754435,-73.976896,NaN,10017,NY,55c2627a498e6734787e29ef
4,Eataly Downtown,"[{'shortName': 'Market', 'pluralName': 'Market...",101 Liberty St,US,New York,United States,btwn Church & Greenwich St,3120,"[101 Liberty St (btwn Church & Greenwich St), ...","[{'lng': -74.011976, 'lat': 40.710075, 'label'...",40.710075,-74.011976,NaN,10007,NY,579c12cd498e6e815b3acf50
5,Shake Shack,"[{'shortName': 'Burgers', 'pluralName': 'Burge...",691 8th Ave,US,New York,United States,at W 44th St,3059,"[691 8th Ave (at W 44th St), New York, NY 1003...",NaN,40.758308,-73.988997,Theater District,10036,NY,4c2bccc7d1a10f475a88f864
6,Eataly,"[{'shortName': 'Gourmet', 'pluralName': 'Gourm...",200 5th Ave,US,New York,United States,at W 23rd St,1256,"[200 5th Ave (at W 23rd St), New York, NY 1001...","[{'lng': -73.989623, 'lat': 40.741997, 'label'...",40.741997,-73.989623,NaN,10010,NY,4c5ef77bfff99c74eda954d3
7,TurnStyle Underground Market,"[{'shortName': 'Food Court', 'pluralName': 'Fo...",1000 S 8th Ave,US,New York,United States,btwn 57th & 58th St,4050,"[1000 S 8th Ave (btwn 57th & 58th St), New Yor...","[{'lng': -73.982569, 'lat': 40.767081, 'label'...",40.767081,-73.982569,NaN,10019,NY,5716655d498e23619f894ef0
8,Hard Rock Cafe New York,"[{'shortName': 'Theme Restaurant', 'pluralName...",1501 Broadway,US,New York,United States,at W 43rd St,2930,"[1501 Broadway (at W 43rd St), New York, NY 10...","[{'lng': -73.98669719696045, 'lat': 40.7571887...",40.757189,-73.986697,NaN,10036,NY,428a8580f964a52083231fe3
9,Del Frisco's Double Eagle Steakhouse,"[{'shortName': 'Steakhouse', 'pluralName': 'St...",1221 Avenue of the Americas,US,New York,United States,at W 49th St,3205,"[1221 Avenue of the Americas (at W 49th St), N...","[{'lng': -73.98222, 'lat': 40.7594099, 'label'...",40.759410,-73.982220,NaN,10020,NY,4b0c24fef964a520be3723e3


In [37]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Katz's Delicatessen,Deli / Bodega,205 E Houston St,US,New York,United States,at Ludlow St,968,"[205 E Houston St (at Ludlow St), New York, NY...",NaN,40.722161,-73.987213,Lower East Side,10002,NY,4acbe67af964a52044c820e3
1,DeKalb Market Hall,Food Court,445 Albee Sq,US,Brooklyn,United States,NaN,4426,"[445 Albee Sq, Brooklyn, NY 11201, United States]","[{'lng': -73.98257926844987, 'lat': 40.6912497...",40.691250,-73.982579,NaN,11201,NY,594277b586f4cc0f251fc389
2,Blue Bottle Coffee,Coffee Shop,1 Rockefeller Plz,US,New York,United States,at W 49th St,3163,"[1 Rockefeller Plz (at W 49th St), New York, N...","[{'lng': -73.978681, 'lat': 40.758545, 'label'...",40.758545,-73.978681,NaN,10020,NY,4f5e82dae4b01989c438cbb6
3,UrbanSpace Vanderbilt,Food Court,230 Park Avenue,US,New York,United States,btwn E 45th St & Vanderbilt Ave,2763,[230 Park Avenue (btwn E 45th St & Vanderbilt ...,"[{'lng': -73.97689591949668, 'lat': 40.7544345...",40.754435,-73.976896,NaN,10017,NY,55c2627a498e6734787e29ef
4,Eataly Downtown,Market,101 Liberty St,US,New York,United States,btwn Church & Greenwich St,3120,"[101 Liberty St (btwn Church & Greenwich St), ...","[{'lng': -74.011976, 'lat': 40.710075, 'label'...",40.710075,-74.011976,NaN,10007,NY,579c12cd498e6e815b3acf50
5,Shake Shack,Burger Joint,691 8th Ave,US,New York,United States,at W 44th St,3059,"[691 8th Ave (at W 44th St), New York, NY 1003...",NaN,40.758308,-73.988997,Theater District,10036,NY,4c2bccc7d1a10f475a88f864
6,Eataly,Gourmet Shop,200 5th Ave,US,New York,United States,at W 23rd St,1256,"[200 5th Ave (at W 23rd St), New York, NY 1001...","[{'lng': -73.989623, 'lat': 40.741997, 'label'...",40.741997,-73.989623,NaN,10010,NY,4c5ef77bfff99c74eda954d3
7,TurnStyle Underground Market,Food Court,1000 S 8th Ave,US,New York,United States,btwn 57th & 58th St,4050,"[1000 S 8th Ave (btwn 57th & 58th St), New Yor...","[{'lng': -73.982569, 'lat': 40.767081, 'label'...",40.767081,-73.982569,NaN,10019,NY,5716655d498e23619f894ef0
8,Hard Rock Cafe New York,Theme Restaurant,1501 Broadway,US,New York,United States,at W 43rd St,2930,"[1501 Broadway (at W 43rd St), New York, NY 10...","[{'lng': -73.98669719696045, 'lat': 40.7571887...",40.757189,-73.986697,NaN,10036,NY,428a8580f964a52083231fe3
9,Del Frisco's Double Eagle Steakhouse,Steakhouse,1221 Avenue of the Americas,US,New York,United States,at W 49th St,3205,"[1221 Avenue of the Americas (at W 49th St), N...","[{'lng': -73.98222, 'lat': 40.7594099, 'label'...",40.759410,-73.982220,NaN,10020,NY,4b0c24fef964a520be3723e3


In [39]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    radius=500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
# type your answer here
NY_venues = getNearbyVenues(names=dataframe_filtered['name'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )

Katz's Delicatessen
DeKalb Market Hall
Blue Bottle Coffee
UrbanSpace Vanderbilt
Eataly Downtown
Shake Shack
Eataly
TurnStyle Underground Market
Hard Rock Cafe New York
Del Frisco's Double Eagle Steakhouse
Junior's Restaurant & Bakery
Starbucks Reserve Roastery
Barnes & Noble
Le District
Starbucks
Han Dynasty
Starbucks (BDFM Subway)
Magnolia Bakery
Starbucks Reserve
Shake Shack
Shake Shack
Bouchon Bakery
Balthazar
Dean & DeLuca
The Halal Guys
Starbucks
Shake Shack
Allegro Coffee Company
Kimbap Lab
Studio Cafe


In [49]:
print(NY_venues.shape)
NY_venues

(2856, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Katz's Delicatessen,40.722161,-73.987213,Russ & Daughters,40.722867,-73.988291,Gourmet Shop
1,Katz's Delicatessen,40.722161,-73.987213,Katz's Delicatessen,40.722161,-73.987213,Deli / Bodega
2,Katz's Delicatessen,40.722161,-73.987213,Il Laboratorio del Gelato,40.722222,-73.987038,Ice Cream Shop
3,Katz's Delicatessen,40.722161,-73.987213,Gaia Italian Cafe,40.721960,-73.985400,Italian Restaurant
4,Katz's Delicatessen,40.722161,-73.987213,Lil' Frankie's,40.723469,-73.988297,Italian Restaurant
5,Katz's Delicatessen,40.722161,-73.987213,ØDD. New York,40.721332,-73.987487,Boutique
6,Katz's Delicatessen,40.722161,-73.987213,Sweet Chick,40.721764,-73.987480,Southern / Soul Food Restaurant
7,Katz's Delicatessen,40.722161,-73.987213,Bar Goto,40.722696,-73.989339,Cocktail Bar
8,Katz's Delicatessen,40.722161,-73.987213,Souvlaki GR,40.721114,-73.987174,Greek Restaurant
9,Katz's Delicatessen,40.722161,-73.987213,Punjabi Grocery & Deli,40.722418,-73.986948,Deli / Bodega


In [48]:
NY_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allegro Coffee Company,100,100,100,100,100,100
Balthazar,100,100,100,100,100,100
Barnes & Noble,100,100,100,100,100,100
Blue Bottle Coffee,100,100,100,100,100,100
Bouchon Bakery,100,100,100,100,100,100
DeKalb Market Hall,100,100,100,100,100,100
Dean & DeLuca,100,100,100,100,100,100
Del Frisco's Double Eagle Steakhouse,100,100,100,100,100,100
Eataly,100,100,100,100,100,100


In [50]:
# one hot encoding
ny_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Colombian Restaurant,Comedy Club,Comic Shop,Community Center,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General College & University,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Strip Club,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,TV Station,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Tonkatsu Restaurant,Tour Provider,Toy / Game Store,Track,Train Station,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Katz's Delicatessen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [52]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Colombian Restaurant,Comedy Club,Comic Shop,Community Center,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General College & University,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Strip Club,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,TV Station,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Tonkatsu Restaurant,Tour Provider,Toy / Game Store,Track,Train Station,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allegro Coffee Company,0.000,0.00,0.000000,0.000000,0.000000,0.030000,0.00,0.00,0.010,0.00,0.01,0.000000,0.00,0.000000,0.00,0.020000,0.000,0.030000,0.020000,0.00,0.000,0.

In [53]:
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allegro Coffee Company----
         venue  freq
0        Hotel  0.08
1  Coffee Shop  0.05
2      Theater  0.05
3  Yoga Studio  0.03
4       Bakery  0.03


----Balthazar----
                venue  freq
0            Boutique  0.06
1      Clothing Store  0.06
2                Café  0.04
3  Italian Restaurant  0.04
4          Shoe Store  0.04


----Barnes & Noble----
            venue  freq
0  Clothing Store  0.05
1           Hotel  0.04
2     Coffee Shop  0.04
3    Cocktail Bar  0.03
4        Boutique  0.03


----Blue Bottle Coffee----
                 venue  freq
0             Boutique  0.04
1       Clothing Store  0.04
2           Steakhouse  0.03
3                Hotel  0.03
4  American Restaurant  0.03


----Bouchon Bakery----
                 venue  freq
0             Boutique  0.04
1       Clothing Store  0.04
2           Steakhouse  0.03
3                Hotel  0.03
4  American Restaurant  0.03


----DeKalb Market Hall----
                venue  freq
0         Coffee Shop  0.07

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allegro Coffee Company,Hotel,Coffee Shop,Theater,Yoga Studio,Bakery,French Restaurant,Food Truck,Park,Cosmetics Shop,American Restaurant
1,Balthazar,Boutique,Clothing Store,Women's Store,Italian Restaurant,Café,Shoe Store,Men's Store,Spa,Seafood Restaurant,Coffee Shop
2,Barnes & Noble,Clothing Store,Coffee Shop,Hotel,Boutique,Cocktail Bar,Sporting Goods Shop,Art Gallery,Women's Store,Food Truck,Spa
3,Blue Bottle Coffee,Clothing Store,Boutique,Chocolate Shop,Food Truck,Steakhouse,American Restaurant,Hotel,Sandwich Place,Japanese Restaurant,Toy / Game Store
4,Bouchon Bakery,Clothing Store,Boutique,Chocolate Shop,Food Truck,Steakhouse,American Restaurant,Hotel,Sandwich Place,Japanese Restaurant,Toy / Game Store
5,DeKalb Market Hall,Coffee Shop,Spa,Sandwich Place,Burger Joint,Italian Restaurant,Pizza Place,Department Store,Bubble Tea Shop,Bagel Shop,Salon / Barbershop
6,Dean & DeLuca,Boutique,Shoe Store,Italian Restaurant,Clothing Store,Spa,Pizza Place,Electronics Store,American Restaurant,Coffee Shop,Mediterranean Restaurant
7,Del Frisco's Double Eagle Steakhouse,Theater,American Restaurant,Chocolate Shop,Food Truck,Hotel,Bakery,Steakhouse,Plaza,Toy / Game Store,Coffee Shop
8,Eataly,Gym,New American Restaurant,Italian Restaurant,Gym / Fitness Center,American Restaurant,Coffee Shop,Lounge,Wine Shop,Sandwich Place,Japanese Restaurant
9,Eataly Downtown,Coffee Shop,Hotel,Clothing Store,Gym,Pizza Place,Women's Store,Sandwich Place,Gym / Fitness Center,Bar,Indian Restaurant


 ## Cluster Neighborhoods

In [57]:
# set number of clusters
kclusters = 5

NY_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 3, 3, 3, 4, 0, 2, 0, 4], dtype=int32)